In [3]:
import pandas as pd
import sqlite3
import logging
from sqlite3 import Error



# Step 1: Fetch the dataset
url = "https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"

data = pd.read_csv(url, sep=';')
data

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Country Code 2,Admin1 Code,Admin2 Code,Admin3 Code,Admin4 Code,Population,Elevation,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates
0,497218,Semikarakorsk,Semikarakorsk,"Semikarakors'k,Semikarakorsk,Semikarakorskaya,...",P,PPLA2,RU,Russian Federation,NaN,61,NaN,NaN,NaN,23583,NaN,13,Europe/Moscow,2019-09-05,Russian Federation,"47.51675,40.80577"
1,498001,Sebezh,Sebezh,"Sebej,Sebesch,Sebez,Sebezas,Sebezh,Sebezj,Sebe...",P,PPLA2,RU,Russian Federation,NaN,60,NaN,NaN,NaN,6688,NaN,132,Europe/Moscow,2019-09-05,Russian Federation,"56.28511,28.48187"
2,498035,Sazonovo,Sazonovo,"Belyye Kresty,Sazonovo,Сазоново",P,PPL,RU,Russian Federation,NaN,85,NaN,NaN,NaN,3509,NaN,136,Europe/Moscow,2012-01-17,Russian Federation,"59.0919,35.2268"
3,499535,Sadovoye,Sadovoye,"Sadovoe,Sadovoje,Sadovoye,Sadowoje,Садовое",P,PPL,RU,Russian Federation,NaN,24,NaN,NaN,NaN,6333,NaN,33,Europe/Moscow,2012-01-17,Russian Federation,"47.7772,44.5208"
4,503682,Putyatino,Putyatino,"Putjanino,Putjatino,Putyatino,Путянино,Путятино",P,PPL,RU,Russian Federation,NaN,62,NaN,NaN,NaN,3044,NaN,128,Europe/Moscow,2019-08-31,Russian Federation,"54.15998,41.1169"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140968,3514222,Yolotepec,Yolotepec,Yolotepec,P,PPL,MX,Mexico,NaN,13,055,NaN,NaN,2487,NaN,1931,America/Mexico_City,2018-11-03,Mexico,"20.38767,-99.07307"
140969,3514292,Yaxe,Yaxe,Yaxe,P,PPLA2,MX,Mexico,NaN,20,561,NaN,NaN,2683,NaN,1505,America/Mexico_City,2018-11-03,Mexico,"16.72595,-96.47191"
140970,3514682,Ciudad Cuauhtémoc,Ciudad Cuauhtemoc,"Cd. Cuauhtemoc,Cd. Cuauhtémoc,Ciudad Cuauhtemo...",P,PPLA2,MX,Mexico,NaN,30,133,NaN,NaN,9740,NaN,9,America/Mexico_City,2020-06-10,Mexico,"22.18439,-97.83472"
140971,3515272,Tonalixco,Tonalixco,NaN,P,PPL,MX,Mexico,NaN,30,185,NaN,NaN,1095,NaN,1425,America/Mexico_City,2013-07-31,Mexico,"18.78194,-97.05972"


In [14]:
# Set up logging
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

# URL of the CSV file
url = "https://public.opendatasoft.com/explore/dataset/geonames-all-cities-with-a-population-1000/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B"

# Function to fetch the dataset from the URL
def fetch_data(url):
    try:
        data = pd.read_csv(url, sep=';')
        logging.info('Data fetched successfully')
        return data
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

# Function to create a connection to the SQLite database
def create_connection():
    conn = None;
    try:
        # Create a connection to a SQLite database in memory
        conn = sqlite3.connect(':memory:')
        logging.info('Database connection created successfully')
    except Error as e:
        logging.error("Exception occurred", exc_info=True)

    return conn

# Function to create a table in the SQLite database
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        # Here we'd enter the SQL commands to create a table
        c.execute(create_table_sql)
        logging.info('Table created successfully')
    except Error as e:
        logging.error("Exception occurred", exc_info=True)

# Function to insert data into the SQLite database
def insert_data(conn, data):
    try:
        # if cities table already exists, we replace it with the new version
        data.to_sql('cities', conn, if_exists='replace', index = False)
        logging.info('Data inserted successfully')
        # Query the database and print the data to verify the insertion
        df = pd.read_sql_query('SELECT * FROM cities', conn)
        print(df)
    except Error as e:
        logging.error("Exception occurred", exc_info=True)

# Function to query the SQLite database to find countries that don't host a megapolis
def query_data(conn):
    try:
        df = pd.read_sql_query('SELECT DISTINCT "Country Code", "Country name EN" FROM cities WHERE population <= 10000000 ORDER BY "Country name EN"', conn)
        logging.info('Data queried successfully')
        return df
    except Error as e:
        logging.error("Exception occurred", exc_info=True)

# Function to save the result in a TSV file
def save_data(df):
    try:
        df.to_csv('countries_without_megapolis.tsv', sep='\t', index=False)
        logging.info('Data saved successfully')
    except Error as e:
        logging.error("Exception occurred", exc_info=True)

# Main function to fetch the data, store it in the database, query the database, and save the result
def main():
    data = fetch_data(url)
    conn = create_connection()
    # Only create table if connection to db is established
    if conn is not None:
        create_table(conn, '''CREATE TABLE IF NOT EXISTS cities (
                                            name text,
                                            "Country code" text
                                            "Country name EN" text,
                                            Population integer
                                        ); ''')
        insert_data(conn, data)
        df = query_data(conn)
        save_data(df)
        
        # Commit changes and close the connection
        conn.commit()
        conn.close()
    else:
        logging.error("Error! cannot create the database connection.")

# Calling main function
if __name__ == "__main__":
    main()


        Geoname ID               Name         ASCII Name  \
0           497218      Semikarakorsk      Semikarakorsk   
1           498001             Sebezh             Sebezh   
2           498035           Sazonovo           Sazonovo   
3           499535           Sadovoye           Sadovoye   
4           503682          Putyatino          Putyatino   
...            ...                ...                ...   
140968     3514222          Yolotepec          Yolotepec   
140969     3514292               Yaxe               Yaxe   
140970     3514682  Ciudad Cuauhtémoc  Ciudad Cuauhtemoc   
140971     3515272          Tonalixco          Tonalixco   
140972     3515404    Tlamixtlahuacan    Tlamixtlahuacan   

                                          Alternate Names Feature Class  \
0       Semikarakors'k,Semikarakorsk,Semikarakorskaya,...             P   
1       Sebej,Sebesch,Sebez,Sebezas,Sebezh,Sebezj,Sebe...             P   
2                         Belyye Kresty,Sazonovo,Сазон

## This program can be exported as .py file and be automated to run weekly using cron